In [84]:
import pandas as pd
import json
import numpy as np

import string
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import pymorphy2
from pymystem3 import Mystem
from stop_words import get_stop_words
from keybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

from tqdm import tqdm

data_path = '../data/'

In [85]:
df = pd.read_csv(data_path + 'prepared_reviews_v3.csv')

In [86]:
df

,index,id,title,text,dateCreate,product,source
0,0,992651,Не выполняют условия акции!,В апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,sravni
1,1,998360,Жалоба на услугу «Газпром Бонус Премиум»,Купил услугу Газпром Бонус «Премиум» за 2 990 ...,2025-09-15T09:38:13.34818Z,debitCards,sravni
2,2,993744,Банк не отвечает за слова своих сотрудников! Н...,"Хочу поделиться историей, которая убила моё до...",2025-09-02T23:21:16.507166Z,debitCards,sravni
3,3,999494,Не выполняют условия акции!,В июне 2025 года я порекомендовал премиальную ...,2025-09-18T08:10:19.954986Z,debitCards,sravni
4,4,992291,Странный банк,Добрый день! В связи с устройством на новую ра...,2025-08-28T19:29:51.696341Z,debitCards,sravni
...,...,...,...,...,...,...,...
50352,50352,10885124,Под видом кредита 12.9% пытаются впарить креди...,Газпромбанк тут заманивал кредитом одобренным ...,2023-04-15 23:33:45,individual,banki
50353,50353,10872392,Неполное изменение номера в офисе/ отделении,Ситуация довольно странная для входа в интерне...,2023-03-30 12:39:01,individual,banki
50354,50354,10852603,Один из отличных банков,В банке нравится все. И обслуживание на отличн...,2023-03-10 15:04:12,individual,banki
50355,50355,10852597,Обращение в контакт центр 10 баллов,"Обращение в контакт центр 10 баллов, а вот раб...",2023-03-10 14:57:02,individual,banki


In [87]:
df['product'].value_counts()

product
debitCards             28300
creditCards             8505
credits                 3950
mortgage                3565
savings                 2744
restructing             1001
serviceLevel             708
autocredits              502
remoteService            365
other                    365
individual               158
creditRefinancing         84
moneyOrder                29
mortgageRefinancing       27
currencyExchange          26
mobile_app                25
usloviya                   3
Name: count, dtype: int64

In [57]:
df  = df[df['product'] != 'debitСards']

# Model

In [88]:
df["review"] = (
    df["title"].fillna('') + " " + df["text"].fillna('')
).str.replace("\xa0", " ", regex=False).str.replace("\n", " ", regex=False)

In [89]:
df

,index,id,title,text,dateCreate,product,source,review
0,0,992651,Не выполняют условия акции!,В апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,sravni,Не выполняют условия акции! В апреле 2025 года...
1,1,998360,Жалоба на услугу «Газпром Бонус Премиум»,Купил услугу Газпром Бонус «Премиум» за 2 990 ...,2025-09-15T09:38:13.34818Z,debitCards,sravni,Жалоба на услугу «Газпром Бонус Премиум» Купил...
2,2,993744,Банк не отвечает за слова своих сотрудников! Н...,"Хочу поделиться историей, которая убила моё до...",2025-09-02T23:21:16.507166Z,debitCards,sravni,Банк не отвечает за слова своих сотрудников! Н...
3,3,999494,Не выполняют условия акции!,В июне 2025 года я порекомендовал премиальную ...,2025-09-18T08:10:19.954986Z,debitCards,sravni,Не выполняют условия акции! В июне 2025 года я...
4,4,992291,Странный банк,Добрый день! В связи с устройством на новую ра...,2025-08-28T19:29:51.696341Z,debitCards,sravni,Странный банк Добрый день! В связи с устройств...
...,...,...,...,...,...,...,...,...
50352,50352,10885124,Под видом кредита 12.9% пытаются впарить креди...,Газпромбанк тут заманивал кредитом одобренным ...,2023-04-15 23:33:45,individual,banki,Под видом кредита 12.9% пытаются впарить креди...
50353,50353,10872392,Неполное изменение номера в офисе/ отделении,Ситуация довольно странная для входа в интерне...,2023-03-30 12:39:01,individual,banki,Неполное изменение номера в офисе/ отделении С...
50354,50354,10852603,Один из отличных банков,В банке нравится все. И обслуживание на отличн...,2023-03-10 15:04:12,individual,banki,Один из отличных банков В банке нравится все. ...
50355,50355,10852597,Обращение в контакт центр 10 баллов,"Обращение в контакт центр 10 баллов, а вот раб...",2023-03-10 14:57:02,individual,banki,Обращение в контакт центр 10 баллов Обращение ...


In [90]:
import pandas as pd
import re

# Твоя функция очистки
def clean_text(text):
    text = re.sub(r'\b\d{5,}\b', '', text)  # большие числа/коды
    text = re.sub(r'\d+\s?(руб|р|₽)', '', text)  # суммы
    text = re.sub(r'\W+', ' ', text)  # спецсимволы
    return text.lower()

# Применяем к колонке 'review'
df['review_clean'] = df['review'].astype(str).apply(clean_text)

# Проверим результат
print(df[['review', 'review_clean']].head())


                                              review  \
0  Не выполняют условия акции! В апреле 2025 года...   
1  Жалоба на услугу «Газпром Бонус Премиум» Купил...   
2  Банк не отвечает за слова своих сотрудников! Н...   
3  Не выполняют условия акции! В июне 2025 года я...   
4  Странный банк Добрый день! В связи с устройств...   

                                        review_clean  
0  не выполняют условия акции в апреле 2025 года ...  
1  жалоба на услугу газпром бонус премиум купил у...  
2  банк не отвечает за слова своих сотрудников не...  
3  не выполняют условия акции в июне 2025 года я ...  
4  странный банк добрый день в связи с устройство...  


In [91]:
# df["review"] = df["title"].fillna('') + " " + df["text"].fillna('')
docs = df["review_clean"].tolist()

In [221]:
docs

['Не выполняют условия акции! В апреле 2025 года я рекомендовала дебетовую карту другу согласно акции банка «Приведи друга». Далее в мае и мною и другом условия были выполнены, но вознаграждение по акции пришло другу только 9 июля 2025, а мне не пришло совсем. Начиная с июня много раз писала в чат поддержки банка, но каждый раз получала стандартные отписки от операторов, что необходимо ждать, и в итоге после долгих мучений другу выплатили вознаграждение 9 июля 2025, как писала ранее. И вот, 15 июля 2025 я снова написала в чат поддержки банка с вопросом почему другу в конце концов пришло вознаграждение, а мне нет, позвала оператора в 12:17, в 12:30 подключился оператор Сергей, попросил 4 цифры телефона друга и сразу отключился, при том что я написала ответ в эту же минуту. Следующий оператор Татьяна ответила в 12:37, написала очередную отписку и отключилась. 04 Августа 2025 оператор Кристина после моего вопроса подключилась и сразу отключилась так и не ответив, оператор София подключила

In [61]:
embedding_model = SentenceTransformer("sberbank-ai/sbert_large_nlu_ru")

In [92]:
embeddings = embedding_model.encode(df["review_clean"].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/1574 [00:00<?, ?it/s]

In [93]:
np.save("../data/reviews_embeddings_v5(re).npy", embeddings)

In [ ]:
# embeddings = np.load("../data/reviews_embeddings_v3.npy")

In [31]:
umap_model = UMAP(
    n_neighbors=25,       # число соседей (можно варьировать)
    n_components=100,      # размерность после сжатия
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

In [39]:
hdbscan_model = HDBSCAN(
    min_cluster_size=150,     # минимум 150 текстов на кластер
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

In [102]:
mystem = Mystem()
punctuation = set(string.punctuation + "«»…—–,()" + ' ,' + ', ')

def lemmatize_text(text):
    """
    Лемматизация текста с Mystem, фильтрация пустых токенов и знаков препинания
    """
    tokens = mystem.lemmatize(text)
    # оставляем только слова (только буквы), приводим к нижнему регистру
    return [t.lower() for t in tokens if t.strip() and not all(c in punctuation for c in t)]

russian_stopwords = get_stop_words("russian")  # из пакета stop-words

vectorizer_model = CountVectorizer(
    tokenizer=lemmatize_text,
    ngram_range=(1,3),     # униграммы + биграммы
    stop_words=russian_stopwords,
    min_df=2
)

In [43]:
topic_model = BERTopic(
    embedding_model=None,      # эмбеддинги мы передаем напрямую
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    language="russian",
    verbose=True
)

In [44]:
topics, probs = topic_model.fit_transform(docs, embeddings)

2025-09-23 00:31:23,087 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-23 00:32:26,114 - BERTopic - Dimensionality - Completed ✓
2025-09-23 00:32:26,119 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-23 00:32:58,115 - BERTopic - Cluster - Completed ✓
2025-09-23 00:32:58,118 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-23 00:34:30,346 - BERTopic - Representation - Completed ✓


In [45]:
topic_info = topic_model.get_topic_info()

In [46]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,93,-1_банк_отец_кредит_сотрудник,"[банк, отец, кредит, сотрудник, мама, газпромб...",[Навязанная кредитная карта и игнорирование пр...
1,0,11750,0_банк_карта_кредитный_кредит,"[банк, карта, кредитный, кредит, счет, газпром...",[Один большой и жирный минус Я держатель зарпл...
2,1,6589,1_банк_карта_кредитный_деньги,"[банк, карта, кредитный, деньги, счет, сотрудн...",[Банк обманул и не вернул обещанный кэш 2000 р...
3,2,4238,2_банк_карта_сотрудник_газпромбанк,"[банк, карта, сотрудник, газпромбанк, вопрос, ...",[Доставка карты газпромбанка Заказывал кредитн...
4,3,546,3_банк_ипотека_документ_квартира,"[банк, ипотека, документ, квартира, кредит, за...",[Рефинансирование ипотечного кредита и некомпе...


# Model V2: BERTopic + KeyBERT

In [94]:
umap_model = UMAP(
    n_neighbors=25,
    n_components=128,   # размерность после сжатия
    min_dist=0.0,
    metric="cosine",
    random_state=42
)
embeddings_umap = umap_model.fit_transform(embeddings)

In [107]:
embeddings

array([[ 0.01114505, -0.00612352, -0.03536812, ..., -0.00489565,
         0.00699366, -0.01057294],
       [ 0.01878096, -0.01603048, -0.00296582, ...,  0.00281675,
        -0.01099384, -0.0273612 ],
       [ 0.03695221,  0.00465995, -0.01718853, ...,  0.00367996,
        -0.0140534 , -0.01605552],
       ...,
       [ 0.00163632, -0.01441602, -0.0402154 , ...,  0.03604563,
        -0.02669396, -0.01420091],
       [ 0.04429065,  0.00767981, -0.00470638, ...,  0.02849353,
        -0.01874949,  0.01292973],
       [-0.00555889,  0.01684343, -0.01674807, ...,  0.00918983,
        -0.0270461 ,  0.01046712]], shape=(50357, 1024), dtype=float32)

In [95]:
np.save("../data/reviews_embeddings_umap_128_re.npy", embeddings_umap)

In [66]:
umap_model = UMAP(
    n_neighbors=25,
    n_components=256,   # размерность после сжатия
    min_dist=0.0,
    metric="cosine",
    random_state=42
)
embeddings_umap = umap_model.fit_transform(embeddings)

In [67]:
np.save("../data/reviews_embeddings_umap_256_noDC.npy", embeddings_umap)

In [ ]:
# hdbscan_model = HDBSCAN(
#     min_cluster_size=25,       # ловим маленькие кластеры
#     min_samples=20,            # плотность точки
#     metric='euclidean',           # для нормализованных эмбеддингов
#     cluster_selection_method='leaf',  # листья иерархии → много мелких кластеров
#     prediction_data=True,
#     allow_single_cluster=False
# )
# hdb_labels = hdbscan_model.fit_predict(embeddings)

# print("Число кластеров (не включая шум):", len(set(hdb_labels)) - (1 if -1 in hdb_labels else 0))
# print("Размеры кластеров:", np.bincount(hdb_labels[hdb_labels >= 0]))
# print("Количество шумовых точек:", np.sum(hdb_labels == -1))

# # Создаем маску для шума
# noise_mask = hdb_labels == -1
# print(f"HDBSCAN: {len(hdb_labels) - noise_mask.sum()} clustered, {noise_mask.sum()} noise points")

In [ ]:
# hdbscan_model = HDBSCAN(
#     min_cluster_size=25,       # ловим маленькие кластеры
#     min_samples=20,            # плотность точки
#     metric='euclidean',           # для нормализованных эмбеддингов
#     cluster_selection_method='leaf',  # листья иерархии → много мелких кластеров
#     prediction_data=True,
#     allow_single_cluster=False
# )

# labels = hdbscan_model.fit_predict(embeddings_umap)

# # Сколько кластеров получилось
# print("Число кластеров (не включая шум):", len(set(labels)) - (1 if -1 in labels else 0))
# print("Размеры кластеров:", np.bincount(labels[labels >= 0]))
# print("Количество шумовых точек:", np.sum(labels == -1))

Число кластеров (не включая шум): 77
Размеры кластеров: [ 33 108 201  68  27  35  34  75  46 170  37 112  97  42  84  76  34  29
  71  34  50  74  34  43  52 420 324  27 208 416 469  30 101  28  52  56
 108  33 124 125 134 280  46 138 110  42  26  74  84  58  35  31 230  40
 179 135 144 196 211  37 125 118  27 133 133 116  35  43 125 187  53  46
  38 823 153 184  47]
Количество шумовых точек: 41554


In [96]:
kmeans_model = KMeans(
    n_clusters=100,
    init='k-means++',
    n_init=30,
    max_iter=500,
    random_state=42,
    algorithm='elkan'
)
labels = kmeans_model.fit_predict(embeddings_umap)

In [10]:
kmeans_model = KMeans(n_clusters=30, random_state=42)
labels = kmeans_model.fit_predict(embeddings_umap)

In [97]:
df['cluster_id'] = labels

In [98]:
from collections import Counter

cluster_counts = Counter(labels)
for cluster_id, count in cluster_counts.items():
    print(f"Кластер {cluster_id}: {count} документов")


Кластер 25: 199 документов
Кластер 18: 303 документов
Кластер 53: 467 документов
Кластер 57: 318 документов
Кластер 31: 278 документов
Кластер 24: 570 документов
Кластер 71: 594 документов
Кластер 79: 484 документов
Кластер 11: 564 документов
Кластер 35: 696 документов
Кластер 59: 539 документов
Кластер 13: 910 документов
Кластер 86: 784 документов
Кластер 68: 180 документов
Кластер 83: 729 документов
Кластер 12: 820 документов
Кластер 66: 630 документов
Кластер 58: 475 документов
Кластер 78: 476 документов
Кластер 20: 743 документов
Кластер 63: 477 документов
Кластер 84: 390 документов
Кластер 32: 540 документов
Кластер 2: 399 документов
Кластер 96: 303 документов
Кластер 43: 524 документов
Кластер 72: 436 документов
Кластер 87: 519 документов
Кластер 3: 489 документов
Кластер 52: 519 документов
Кластер 76: 481 документов
Кластер 45: 631 документов
Кластер 22: 430 документов
Кластер 97: 589 документов
Кластер 73: 414 документов
Кластер 23: 531 документов
Кластер 36: 255 документов
Кла

In [99]:
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
russian_stopwords_nltk = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /home/gleb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
from stop_words import get_stop_words

russian_stopwords = get_stop_words("russian")

In [101]:
russian_stopwords = russian_stopwords + russian_stopwords_nltk + ['газпромбанк']

### Центроиды берем из кандидадтов

In [105]:
cluster_keywords = {}
cluster_keywords_sims = {} 

for cluster_id in tqdm(range(100), desc="Обработка кластеров"):
    # Все документы из текущего кластера
    cluster_texts = [docs[i] for i, label in enumerate(labels) if label == cluster_id]
    if not cluster_texts:
        continue

    # 🔹 2. Отбор кандидатов через CountVectorizer
    vectorizer = CountVectorizer(
        tokenizer = lemmatize_text,
        ngram_range=(1,1),      # униграммы и биграммы
        min_df=2,
        max_df=0.96,                
        stop_words=russian_stopwords    # убираем стоп-слова
    )
    vectorizer.fit(cluster_texts)
    candidates = vectorizer.get_feature_names_out()
    if len(candidates) == 0:
        print(f'Cluster_id {cluster_id}, len(candidates) == 0')
        cluster_keywords[cluster_id] = []
        continue

    # 🔹 3. Считаем центроид кластера
    doc_embs = embedding_model.encode(cluster_texts, convert_to_numpy=True, show_progress_bar=False)
    centroid = doc_embs.mean(axis=0)

    # 🔹 4. Эмбеддинги кандидатов
    cand_embs = embedding_model.encode(candidates, convert_to_numpy=True, show_progress_bar=False)

    # 🔹 5. Косинусная близость кандидатов к центроиду
    sims = cosine_similarity([centroid], cand_embs)[0]

    # 🔹 6. Топ-10 кандидатов
    top_idx = sims.argsort()[::-1][:10]
    top_candidates = [candidates[i] for i in top_idx]
    top_sims = [sims[i] for i in top_idx]

    cluster_keywords[cluster_id] = top_candidates
    cluster_keywords_sims[cluster_id] = top_sims  # сохраняем веса для топ-10 ключевых слов

Обработка кластеров: 100%|██████████| 100/100 [11:04<00:00,  6.65s/it]


In [106]:
for cluster_id, kws in cluster_keywords.items():
    print(f"Кластер {cluster_id}: {kws}")

Кластер 0: ['автокредит', 'ипотечный', 'рефинансирование', 'газпромбанк', 'нотариальный', 'рассрочка', 'залоговый', 'погашение', 'закладная', 'кредитование']
Кластер 1: ['клиентоориентированность', 'переоформление', 'газпромбанк', 'клиентоориентированный', 'кэшбэк', 'кэшбек', 'госуслуга', 'рефинансирование', 'кредитный', 'кредитка']
Кластер 2: ['транзакция', 'юникредит', 'авторизация', 'переоформление', 'кэшбек', 'кэшбэк', 'клиентоориентированность', 'промокод', 'тарифный', 'газпромбанк']
Кластер 3: ['транзакция', 'кэшбэк', 'кредитный', 'газпромбанк', 'кэшбек', 'кредитка', 'сбербанк', 'сбера', 'газпробанк', 'платежный']
Кластер 4: ['транзакция', 'нотариальный', 'обжалование', 'автокредит', 'правоприменительный', 'рефинансирование', 'погашение', 'банкротство', 'росфинмониторинг', 'газпромбанк']
Кластер 5: ['автокредит', 'потребкредит', 'ингосстрах', 'юникредит', 'транзакция', 'промсвязьбанк', 'госуслуга', 'рефинанс', 'газромбанк', 'газпромбанк']
Кластер 6: ['сберсчет', 'платежка', 'потр

In [104]:
for cluster_id, kws in cluster_keywords.items():
    print(f"Кластер {cluster_id}: {kws}")

Кластер 0: ['банк одобрение ипотечный', 'оформление ипотека газпромбанк', 'заявление рефинансирование ипотека', 'заявка рефинансирование ипотека', 'рефинансирование ипотека оформление', 'заявка ипотека газпромбанк', 'первичный сделка ипотека', 'заявка ипотечный кредит', 'перекредитовать переплата ипотека', 'досрочный погашение ипотека']
Кластер 1: ['банк оформление дебетовый', 'оформление карта дебетовый', 'перевыпуск дебетовый карта', 'банка клиентоориентированность итог', 'получение карта дебетовый', 'газпромбанк получение дебетовый', 'повод кредитный дебетовый', 'довольный представитель банк', 'оформление использование дебетовый', 'оформление дебетовый карта']
Кластер 2: ['списание активированный кредитка', 'судьба начисление кэшбэк', 'банк сервис разблокировка', 'ошибочный списание дебетовый', 'навязывание кредитный карта', 'смс информирование банк', 'карточка юнионпеивать дебетовый', 'начисление кэшбэк периодически', 'отзыв сервис банк', 'банка начисление кэшбэк']
Кластер 3: ['неп

In [80]:
df[df['cluster_id'] == 93]

,index,id,title,text,dateCreate,product,source,review,cluster_id
4,4,992291,Странный банк,Добрый день! В связи с устройством на новую ра...,2025-08-28T19:29:51.696341Z,debitCards,sravni,Странный банк Добрый день! В связи с устройств...,93
68,68,955229,Газпромбанк — настоящие м*******!,"Банк, который не платит бонусы за приглашения....",2025-05-09T11:58:30.137327Z,debitCards,sravni,"Газпромбанк — настоящие м*******! Банк, которы...",93
335,335,585976,Опыт получения зарплатной карты в Газпромбанке...,"Для того, чтобы получить зарплатную карту в Га...",2023-01-24T02:17:44.52552Z,debitCards,sravni,Опыт получения зарплатной карты в Газпромбанке...,93
398,398,613368,Все акции - ложь и обман! Мошенники!,Акция Дебетовая карта «Мир»\n\n500 ₽ за оформл...,2023-06-05T14:31:43.370865Z,debitCards,sravni,Все акции - ложь и обман! Мошенники! Акция Деб...,93
471,471,969848,Хуже микрозаймов!,"Не доверяйте свой деньги этой конторе, они сво...",2025-06-23T09:20:52.53055Z,debitCards,sravni,Хуже микрозаймов! Не доверяйте свой деньги это...,93
...,...,...,...,...,...,...,...,...,...
49989,49989,11856877,Хитрый способ начисления процентов по накопите...,В Газпромбанке хитрый способ начисления процен...,2024-11-03 08:48:23,savings,banki,Хитрый способ начисления процентов по накопите...,93
50028,50028,11852188,Обходите этот банк стороной! Обман клиентов!,Собирались с супругой вложить деньги на сьерег...,2024-11-01 18:42:12,savings,banki,Обходите этот банк стороной! Обман клиентов! С...,93
50189,50189,12542122,Вклад,Два вклада: с возможностью снятия и пополнения...,2025-09-10 17:10:30,mobile_app,banki,Вклад Два вклада: с возможностью снятия и попо...,93
50201,50201,12370447,5 из 5,. . ...,2025-06-20 12:53:56,individual,banki,5 из 5 . . ...,93


In [ ]:
for cluster_id, kws in cluster_keywords.items():
    print(f"Кластер {cluster_id}: {kws}")

Кластер 0: ['оформил дебетовую', 'ответ банка', 'оформил кредитную', 'оформление дебетовой', 'маркетинговой акции', 'вознаграждение акции', 'получил дебетовую', 'кэшбэк 35', 'unionpay газпромбанка', 'накопительный счёт']
Кластер 1: ['банк заблокировал', 'оформила кредитную', 'обратилась банк', 'оформила дебетовую', 'открыла вклад', 'звоню банк', 'смс банка', 'банк списал', 'позвонила банк', 'карту заблокировали']
Кластер 2: ['газпромбанк отличный', 'отличный банк', 'банк отличный', 'надежный банк', 'надёжный банк', 'банк газпромбанк', 'рекомендую банк', 'удобный банк', 'рекомендую газпромбанк', 'потребительский кредит']
Кластер 3: ['ответ банка', 'оформил дебетовую', 'оформил кредитную', 'обратился банк', 'приложение банка', 'unionpay газпромбанка', 'кредитный лимит', 'получил дебетовую', 'накопительный счёт', 'накопительный счет']
Кластер 4: ['закрытие счета', 'списание средств', 'банк нарушает', 'возврат денежных', 'кредитный договор', 'списание денежных', 'досрочное погашение', 'спи

In [ ]:
for cluster_id, sim in cluster_keywords_sims.items():
    print(f"Кластер {cluster_id}: {sim}")

Кластер 0: [np.float32(0.554049), np.float32(0.554049), np.float32(0.52324057), np.float32(0.486225), np.float32(0.48457852), np.float32(0.48223093), np.float32(0.47827435), np.float32(0.4765326), np.float32(0.4751514), np.float32(0.47433782)]
Кластер 1: [np.float32(0.52899563), np.float32(0.5110281), np.float32(0.5086787), np.float32(0.503484), np.float32(0.4956783), np.float32(0.49508676), np.float32(0.49283746), np.float32(0.4890301), np.float32(0.47918576), np.float32(0.47785687)]
Кластер 2: [np.float32(0.5434723), np.float32(0.5275197), np.float32(0.50972295), np.float32(0.49310336), np.float32(0.49108303), np.float32(0.49108303), np.float32(0.47702816), np.float32(0.44961113), np.float32(0.44850254), np.float32(0.4477152)]
Кластер 3: [np.float32(0.49076462), np.float32(0.47473073), np.float32(0.46495593), np.float32(0.46325797), np.float32(0.45569515), np.float32(0.45023626), np.float32(0.4490807), np.float32(0.44222263), np.float32(0.4416638), np.float32(0.43373406)]
Кластер 4: 

### Центроиды - это уже готовые продукты

In [24]:
with open("categories.json", "r", encoding="utf-8") as f:
    categories_data = json.load(f)

# Строим список центроидов
centroid_labels = []
for cat, val in categories_data.items():
    subs = val.get("subcategories", [])
    if subs:
        centroid_labels.extend([f"{sub} {cat}" for sub in subs])
    else:
        centroid_labels.append(cat)


len(centroid_labels)

34

In [25]:
# Считаем эмбеддинги центроидов
# embedding_model = SentenceTransformer("sberbank-ai/sbert_large_nlu_ru")
centroid_embs = embedding_model.encode(centroid_labels, convert_to_numpy=True, show_progress_bar=True)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
SIM_THRESHOLD = 0.40  

results = {}
topic_counts = defaultdict(int)  

for cluster_id, keywords in tqdm(cluster_keywords.items(), desc="Сравнение кластеров"):
    if not keywords:
        continue

    # эмбеддинги ключевых слов кластера
    keyword_embs = embedding_model.encode(keywords, convert_to_numpy=True, show_progress_bar=False)
    cluster_centroid = keyword_embs.mean(axis=0)

    # косинусная близость с предопределёнными центроидами
    sims = cosine_similarity([cluster_centroid], centroid_embs)[0]

    # топ-3 ближайших
    top_idx = sims.argsort()[::-1][:3]
    top_labels = [centroid_labels[i] for i in top_idx]
    top_sims = [sims[i] for i in top_idx]

    if max(top_sims) < SIM_THRESHOLD:
        results[cluster_id] = {
            "status": "❌ Близких кластеров нет",
            "keywords": keywords
        }
    else:
        results[cluster_id] = {
            "status": "✅ Найдены ближайшие",
            "matches": list(zip(top_labels, top_sims)),
            "keywords": keywords
        }
        # считаем только наиболее близкий топик (первый)
        topic_counts[top_labels[0]] += 1  

Сравнение кластеров: 100%|██████████| 100/100 [00:02<00:00, 49.41it/s]


In [40]:
# Пример вывода
for cid, info in results.items():
    print(f"\n=== Кластер {cid} ===")
    print(info["status"])
    if "matches" in info:
        for label, score in info["matches"]:
            print(f"  {label}: {score:.3f}")
    print("Ключевые слова:", ", ".join(info["keywords"]))


=== Кластер 0 ===
✅ Найдены ближайшие
  Автокредит Автокредиты: 0.864
  Рефинансирование ипотеки: 0.848
  Потребительский кредит Кредиты наличными: 0.843
Ключевые слова: отказ банк, обратился сбербанк, ипотеке просрочка, отказы кредитовании, заявка автокредит, заявление рефинансирование, платеж ипотечный, обратился автокредитом, задолженность 2млн, заявку ипотечный

=== Кластер 1 ===
✅ Найдены ближайшие
  Потребительский кредит Кредиты наличными: 0.724
  Под залог недвижимости Кредиты наличными: 0.717
  Накопительный Вклады: 0.702
Ключевые слова: заблокировали дебетовую, активировала кредитную, платеж заблокировали, счет заблокирован, банк заблокировал, банк проконсультировалась, заблокировали счет, оформляла кредитную, счет закрыла, выпустила кредитную

=== Кластер 2 ===
✅ Найдены ближайшие
  Дистанционное обслуживание Обслуживание: 0.797
  Обслуживание в офисе Обслуживание: 0.794
  Ежедневный процент Вклады: 0.774
Ключевые слова: клиентоориентированность оперативность, благодарю бан

In [41]:
print("\n=== Количество кластеров по топикам ===")
for label in centroid_labels:  # выводим все топики, даже если 0
    print(f"{label}: {topic_counts[label]}")


=== Количество кластеров по топикам ===
Умная карта Дебетовые карты: 1
Умная карта (Премиум) Дебетовые карты: 0
Пенсионная карта Дебетовые карты: 1
Дебетовая карта Дебетовые карты: 2
Кредитная карта 90 дней Кредитные карты: 0
Простая кредитная карта Кредитные карты: 0
Кредитная карта Кредитные карты: 1
Под залог недвижимости Кредиты наличными: 7
Под залог имеющегося авто Кредиты наличными: 0
Потребительский кредит Кредиты наличными: 54
Новые деньги Вклады: 0
Большая выгода Вклады: 0
Ежедневный процент Вклады: 11
Накопительный Вклады: 1
Вклад Вклады: 0
Вторичное жилье Ипотека: 0
Ипотека на дом Ипотека: 0
На новостройку Ипотека: 2
Ипотека на строительство дома Ипотека: 0
На автомобиль Автокредиты: 0
На покупку машины в автосалоне Автокредиты: 0
Автокредит Автокредиты: 4
Реструктуризация Рефинансирование кредитов: 4
Рефинансирование Рефинансирование кредитов: 2
Рефинансирование ипотеки: 6
Обмен валют: 0
Условия: 0
Денежные переводы: 0
Обслуживание в офисе Обслуживание: 1
Дистанционное об

In [44]:
mapped_labels = [f"{sub} {cat}" for cat, sub in mapping.values()]

# Категории, которых нет в mapping
missing = set(centroid_labels) - set(mapped_labels)

print("Категории без мапинга:")
for m in missing:
    print("-", m)

Категории без мапинга:
- Другие Другое
- Новые деньги Вклады
- Ипотека на строительство дома Ипотека
- Вторичное жилье Ипотека
- Большая выгода Вклады
- Условия
- Ежедневный процент Вклады
- Пенсионная карта Дебетовые карты
- Умная карта (Премиум) Дебетовые карты
- Обмен валют
- Денежные переводы
- Мобильное приложение
- На автомобиль Автокредиты
- Простая кредитная карта Кредитные карты
- На покупку машины в автосалоне Автокредиты
- Накопительный Вклады
- Рефинансирование ипотеки
- Кредитная карта 90 дней Кредитные карты
- Под залог недвижимости Кредиты наличными
- На новостройку Ипотека
- Под залог имеющегося авто Кредиты наличными
- Умная карта Дебетовые карты


In [47]:
mapping_full = {
    # ====== ДЕБЕТОВЫЕ КАРТЫ ======
    "debitCards": [
        "Дебетовая карта",
        "Умная карта",
        "Умная карта (Премиум)",
        "Пенсионная карта"
    ],

    # ====== КРЕДИТНЫЕ КАРТЫ ======
    "creditCards": [
        "Кредитная карта",
        "Кредитная карта 90 дней",
        "Простая кредитная карта"
    ],

    # ====== КРЕДИТЫ НАЛИЧНЫМИ ======
    "credits": [
        "Потребительский кредит",
        "Под залог недвижимости",
        "Под залог имеющегося авто"
    ],

    # ====== ВКЛАДЫ ======
    "savings": [
        "Вклад",
        "Новые деньги",
        "Большая выгода",
        "Ежедневный процент",
        "Накопительный"
    ],

    # ====== ИПОТЕКА ======
    "mortgage": [
        "Ипотека на дом",
        "Вторичное жилье",
        "На новостройку",
        "Ипотека на строительство дома"
    ],

    # ====== АВТОКРЕДИТЫ ======
    "autocredits": [
        "Автокредит",
        "На автомобиль",
        "На покупку машины в автосалоне"
    ],

    # ====== РЕФИНАНСИРОВАНИЕ КРЕДИТОВ ======
    "restructing": [
        "Реструктуризация"
    ],
    "creditRefinancing": [
        "Рефинансирование"
    ],

    # ====== РЕФИНАНСИРОВАНИЕ ИПОТЕКИ ======
    "mortgageRefinancing": [
        "Рефинансирование ипотеки"
    ],

    # ====== ОБМЕН ВАЛЮТ ======
    "currencyExchange": [
        "Обмен валют"
    ],

    # ====== ДЕНЕЖНЫЕ ПЕРЕВОДЫ ======
    "moneyOrder": [
        "Денежные переводы"
    ],

    # ====== ОБСЛУЖИВАНИЕ ======
    "serviceLevel": [
        "Обслуживание в офисе"
    ],
    "remoteService": [
        "Дистанционное обслуживание"
    ],

    # ====== МОБИЛЬНОЕ ПРИЛОЖЕНИЕ ======
    "mobile_app": [
        "Мобильное приложение"
    ],

    # ====== УСЛОВИЯ ======
    "usloviya": [
        "Условия"
    ],

    # ====== ДРУГОЕ ======
    "other": [
        "Другие услуги"
    ],
    "individual": [
        "Другое (физ. лица)"
    ],

    # ====== OTHERS (для непопавших) ======
    "OTHERS": [
        "Другие"
    ]
}


In [50]:
from collections import defaultdict

# --- 1) Воссоздаем centroid_labels -> (sub, cat) map (точно так же, как ты строил centroid_labels) ---
centroid_to_parts = {}
centroid_labels = []
for cat, val in categories_data.items():
    subs = val.get("subcategories", [])
    if subs:
        for sub in subs:
            label = f"{sub} {cat}"
            centroid_labels.append(label)
            centroid_to_parts[label] = (sub, cat)
    else:
        label = cat
        centroid_labels.append(label)
        centroid_to_parts[label] = (None, cat)

# --- 2) Убедимся, что в mapping_full есть ключ OTHERS для "непопавших" ---
if "OTHERS" not in mapping_full:
    mapping_full["OTHERS"] = ["Другие"]

# --- 3) Подготовим контейнеры для результатов ---
tag_counts = {tag: 0 for tag in mapping_full.keys()}
tag_details = {tag: [] for tag in mapping_full.keys()}  # какие centroid_labels внесли вклад



In [51]:

# topic_counts: словарь { "Подкатегория Категория": count }  (как у тебя ранее)
# Если topic_counts отсутствует — замените на {} или посчитайте заранее.
topic_counts = globals().get("topic_counts", {})  # предполагаем, что он существует

# --- 4) Маппинг: для каждого centroid_label ищем соответствующий тег ---
for label in centroid_labels:
    cnt = int(topic_counts.get(label, 0))  # сколько кластеров у этой подкатегории
    sub, cat = centroid_to_parts[label]

    matched_tag = None

    # 4.1 сначала ищем прямое совпадение по подкатегории (самая строгая логика)
    if sub:
        for tag, sublist in mapping_full.items():
            if sub in sublist:
                tag_counts[tag] += cnt
                if cnt:
                    tag_details[tag].append((label, cnt))
                matched_tag = tag
                break

    # 4.2 если не нашли по подкатегории — попробуем матч по категории или по полной метке
    if matched_tag is None:
        for tag, sublist in mapping_full.items():
            if cat in sublist or label in sublist:
                tag_counts[tag] += cnt
                if cnt:
                    tag_details[tag].append((label, cnt))
                matched_tag = tag
                break

    # 4.3 если всё ещё не найдено — кладём в OTHERS
    if matched_tag is None:
        tag_counts["OTHERS"] += cnt
        if cnt:
            tag_details["OTHERS"].append((label, cnt))

# --- 5) Вывод результатов ---
print("=== Количество кластеров по тегам (mapping_full keys) ===")
for tag in tag_counts:
    print(f"{tag}: {tag_counts[tag]}")

# Подробно: какие centroid_labels внесли вклад в каждый тег (только если есть вклад)
print("\n=== Детали (какие подкатегории дали кластеры) ===")
for tag, items in tag_details.items():
    if not items:
        continue
    print(f"\n{tag} (всего {tag_counts[tag]}):")
    for label, c in items:
        print(f"  {label}: {c}")

=== Количество кластеров по тегам (mapping_full keys) ===
debitCards: 4
creditCards: 1
credits: 61
savings: 12
mortgage: 2
autocredits: 4
restructing: 4
creditRefinancing: 2
mortgageRefinancing: 6
currencyExchange: 0
moneyOrder: 0
serviceLevel: 1
remoteService: 3
mobile_app: 0
usloviya: 0
other: 0
individual: 0
OTHERS: 0

=== Детали (какие подкатегории дали кластеры) ===

debitCards (всего 4):
  Умная карта Дебетовые карты: 1
  Пенсионная карта Дебетовые карты: 1
  Дебетовая карта Дебетовые карты: 2

creditCards (всего 1):
  Кредитная карта Кредитные карты: 1

credits (всего 61):
  Под залог недвижимости Кредиты наличными: 7
  Потребительский кредит Кредиты наличными: 54

savings (всего 12):
  Ежедневный процент Вклады: 11
  Накопительный Вклады: 1

mortgage (всего 2):
  На новостройку Ипотека: 2

autocredits (всего 4):
  Автокредит Автокредиты: 4

restructing (всего 4):
  Реструктуризация Рефинансирование кредитов: 4

creditRefinancing (всего 2):
  Рефинансирование Рефинансирование кр

In [52]:
df['product'].value_counts()

product
debitCards             28300
creditCards             8505
credits                 3950
mortgage                3565
savings                 2744
restructing             1001
serviceLevel             708
autocredits              502
remoteService            365
other                    365
individual               158
creditRefinancing         84
moneyOrder                29
mortgageRefinancing       27
currencyExchange          26
mobile_app                25
usloviya                   3
Name: count, dtype: int64

## Ищем ТОП-3 Темы по полученным Key-Words

In [107]:
SIMILARITY_THRESHOLD = 0.1  # ниже — считаем кластер неопределённым
TOP_K = 3  # сколько категорий выдавать

In [164]:
with open("categories.json", "r", encoding="utf-8") as f:
    categories_json = json.load(f)

In [186]:
category_strings = []   # строки для эмбеддингов
category_map = []       # хранит верхнюю категорию каждой строки

for main_cat, val in categories_json.items():
    # Сначала добавляем саму главную категорию
    category_strings.append(main_cat)
    category_map.append(main_cat)

    # Добавляем подкатегории, если они есть
    if "subcategories" in val and val["subcategories"]:
        for subcat in val["subcategories"]:
            category_strings.append(f"{main_cat} {subcat}")
            category_map.append(main_cat)

print(f"Всего категорий и подкатегорий для эмбеддингов: {len(category_strings)}")


category_embeddings = embedding_model.encode(category_strings, convert_to_numpy=True, show_progress_bar=True)


Всего категорий и подкатегорий для эмбеддингов: 44


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [168]:
category_strings

['Физические лица',
 'Дебетовые карты',
 'Дебетовые карты Умная карта',
 'Дебетовые карты Умная карта (Премиум)',
 'Дебетовые карты Пенсионная карта',
 'Дебетовые карты Дебетовая карта',
 'Кредитные карты',
 'Кредитные карты Кредитная карта 90 дней',
 'Кредитные карты Простая кредитная карта',
 'Кредитные карты Кредитная карта',
 'Кредиты наличными',
 'Кредиты наличными Под залог недвижимости',
 'Кредиты наличными Под залог имеющегося авто',
 'Кредиты наличными Потребительский кредит',
 'Вклады',
 'Вклады Новые деньги',
 'Вклады Большая выгода',
 'Вклады Ежедневный процент',
 'Вклады Накопительный',
 'Вклады Вклад',
 'Ипотека',
 'Ипотека Вторичное жилье',
 'Ипотека Ипотека на дом',
 'Ипотека На новостройку',
 'Ипотека Ипотека на строительство дома',
 'Автокредиты',
 'Автокредиты На автомобиль',
 'Автокредиты На покупку машины в автосалоне',
 'Автокредиты Автокредит',
 'Рефинансирование кредитов',
 'Рефинансирование кредитов Реструктуризация',
 'Рефинансирование кредитов Рефинансировани

In [171]:
category_strings[18]

'Вклады Накопительный'

In [ ]:
# category_embeddings = embedding_model.encode(all_category_names, convert_to_numpy=True, show_progress_bar=True)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [205]:
cluster_to_categories = {}

for cluster_id, keywords in tqdm(cluster_keywords.items(), desc="Обработка кластеров"):
    if not keywords:
        print(f'Empty cluster: {cluster_id}')
        cluster_to_categories[cluster_id] = {
            "top_categories": [],
            "status": "Пустой кластер"
        }
        continue

    # Эмбеддинги ключевых слов
    key_embs = embedding_model.encode(keywords[:3], convert_to_numpy=True, show_progress_bar=False)

    # Веса из ранее сохранённой sims
    weights = np.array(cluster_keywords_sims[cluster_id][:3])
    weights = weights / weights.sum()  # нормируем на 1

    cluster_emb = np.average(key_embs, axis=0).reshape(1, -1)
    sims_to_categories = cosine_similarity(cluster_emb, category_embeddings)[0]

    # cluster_emb = embedding_model.encode(keywords, convert_to_numpy=True, show_progress_bar=False)
    # cluster_emb = cluster_emb.mean(axis=0).reshape(1, -1)  # 2D для cosine_similarity

    # # косинусная близость к категориям
    # sims = cosine_similarity(cluster_emb, category_embeddings)[0]

    # топ-K категорий
    top_idx = sims_to_categories.argsort()[::-1][:TOP_K]
    top_categories = [all_category_names[i] for i in top_idx]

    # проверка порога
    if sims[top_idx[0]] < SIMILARITY_THRESHOLD:
        status = "Неопределённый / новый топик"
    else:
        status = "Определённый"

    cluster_to_categories[cluster_id] = {
        "top_categories": top_categories,
        "top_scores": [float(sims[i]) for i in top_idx],
        "status": status
    }



Обработка кластеров: 100%|██████████| 30/30 [00:00<00:00, 33.34it/s]


In [190]:
cluster_keywords[0][:3]

['накопительный счет', 'накопительный счёт', 'накопительного счета']

In [ ]:
category_embeddings[18]

array([ 0.01286559, -0.02260381, -0.0448326 , ..., -0.00831546,
       -0.04883856,  0.03100483], shape=(1024,), dtype=float32)

In [191]:
key_embs_0 = embedding_model.encode(cluster_keywords[0][:3], convert_to_numpy=True, show_progress_bar=False) 

In [180]:
key_embs_0

array([[ 1.0833771e-02, -2.5976408e-02, -5.0636396e-02, ...,
         8.5316151e-03,  8.6662447e-05,  6.9430857e-03],
       [ 1.0833771e-02, -2.5976408e-02, -5.0636396e-02, ...,
         8.5316151e-03,  8.6662447e-05,  6.9430857e-03],
       [ 1.7893482e-02, -2.5537888e-02, -2.9878417e-02, ...,
         3.5063010e-03, -2.6663324e-02,  1.3148412e-02],
       ...,
       [ 2.8961677e-02, -2.7739955e-02, -5.7651002e-02, ...,
         9.6294163e-03, -1.1259182e-03, -1.2336349e-03],
       [ 3.1138498e-02, -9.9402620e-03, -5.2917678e-02, ...,
        -2.3787063e-05, -4.2648003e-02,  4.0554511e-03],
       [ 7.8277607e-03, -9.3518449e-03, -3.0167159e-02, ...,
        -1.0463889e-03, -2.6664609e-02,  3.5294406e-02]],
      shape=(10, 1024), dtype=float32)

In [192]:
weights = np.array(cluster_keywords_sims[0])
weights = weights / weights.sum()  # нормируем на 1

cluster_emb_0 = np.average(key_embs_0, axis=0).reshape(1, -1)

In [193]:
cosine_similarity(cluster_emb_0, category_embeddings)

array([[0.48770976, 0.7656184 , 0.6638851 , 0.6797792 , 0.79994464,
        0.7637166 , 0.7650412 , 0.75643814, 0.7275584 , 0.73435843,
        0.82753897, 0.79530364, 0.76972246, 0.80275786, 0.6965653 ,
        0.7136996 , 0.5972873 , 0.86312264, 0.8339992 , 0.79541695,
        0.63600993, 0.6193162 , 0.72818124, 0.70917827, 0.69196564,
        0.6903969 , 0.7564481 , 0.70320743, 0.6700062 , 0.7296804 ,
        0.7212677 , 0.72836614, 0.7229326 , 0.6403917 , 0.4789811 ,
        0.74008846, 0.6457393 , 0.64578754, 0.6548135 , 0.29240328,
        0.252775  , 0.39196855, 0.3301695 , 0.55257905]], dtype=float32)

In [194]:
cosine_similarity(cluster_emb_0, category_embeddings).argsort()[::-1][:TOP_K]

array([[40, 39, 42, 41, 34,  0, 43, 16, 21, 20, 33, 36, 37, 38,  2, 28,
         3, 25, 24, 14, 27, 23, 15, 30, 32,  8, 22, 31, 29,  9, 35,  7,
        26,  5,  6,  1, 12, 11, 19,  4, 13, 10, 18, 17]])

In [197]:
category_strings[10]

'Кредиты наличными'

In [161]:
cosine_similarity(cluster_emb_0, [category_embeddings[18]])

array([[0.7371304]], dtype=float32)

In [162]:
cosine_similarity(cluster_emb_0, [category_embeddings[10]])

array([[0.82753897]], dtype=float32)

In [206]:
for cluster_id, info in cluster_to_categories.items():
    print(f"Кластер {cluster_id} ({info['status']}): {info['top_categories']} с оценками {info['top_scores']}")

Кластер 0 (Определённый): ['Ежедневный процент', 'Накопительный', 'Кредиты наличными'] с оценками [0.35855019092559814, 0.298113077878952, 0.27568092942237854]
Кластер 1 (Определённый): ['Обслуживание в офисе', 'Дистанционное обслуживание', 'Денежные переводы'] с оценками [0.2045556604862213, 0.21177983283996582, 0.2690359950065613]
Кластер 2 (Определённый): ['Кредиты наличными', 'Автокредит', 'Потребительский кредит'] с оценками [0.27568092942237854, 0.20571684837341309, 0.2250657081604004]
Кластер 3 (Определённый): ['Потребительский кредит', 'Кредиты наличными', 'Под залог недвижимости'] с оценками [0.2250657081604004, 0.27568092942237854, 0.2768118977546692]
Кластер 4 (Определённый): ['Рефинансирование ипотеки', 'Рефинансирование кредитов', 'Рефинансирование'] с оценками [0.18984343111515045, 0.18972845375537872, 0.3100571036338806]
Кластер 5 (Определённый): ['Кредиты наличными', 'Кредитные карты', 'Пенсионная карта'] с оценками [0.27568092942237854, 0.28399384021759033, 0.317020177

In [202]:
for cluster_id, info in cluster_to_categories.items():
    print(f"Кластер {cluster_id} ({info['status']}): {info['top_categories']} с оценками {info['top_scores']}")

Кластер 0 (Определённый): ['Ежедневный процент', 'Накопительный', 'Кредиты наличными'] с оценками [0.35855019092559814, 0.298113077878952, 0.27568092942237854]
Кластер 1 (Определённый): ['Обслуживание в офисе', 'Дистанционное обслуживание', 'Денежные переводы'] с оценками [0.2045556604862213, 0.21177983283996582, 0.2690359950065613]
Кластер 2 (Определённый): ['Кредиты наличными', 'Автокредит', 'Потребительский кредит'] с оценками [0.27568092942237854, 0.20571684837341309, 0.2250657081604004]
Кластер 3 (Определённый): ['Потребительский кредит', 'Кредиты наличными', 'Под залог недвижимости'] с оценками [0.2250657081604004, 0.27568092942237854, 0.2768118977546692]
Кластер 4 (Определённый): ['Рефинансирование ипотеки', 'Рефинансирование кредитов', 'Рефинансирование'] с оценками [0.18984343111515045, 0.18972845375537872, 0.3100571036338806]
Кластер 5 (Определённый): ['Кредиты наличными', 'Кредитные карты', 'Пенсионная карта'] с оценками [0.27568092942237854, 0.28399384021759033, 0.317020177

In [109]:
for cluster_id, info in cluster_to_categories.items():
    print(f"Кластер {cluster_id} ({info['status']}): {info['top_categories']} с оценками {info['top_scores']}")

Кластер 0 (Определённый): ['Кредиты наличными', 'Потребительский кредит', 'Кредитная карта'] с оценками [0.27568092942237854, 0.2250657081604004, 0.25561290979385376]
Кластер 1 (Определённый): ['Кредитная карта', 'Дебетовая карта', 'Пенсионная карта'] с оценками [0.25561290979385376, 0.2906298339366913, 0.3170201778411865]
Кластер 2 (Определённый): ['Кредиты наличными', 'Кредитная карта', 'Потребительский кредит'] с оценками [0.27568092942237854, 0.25561290979385376, 0.2250657081604004]
Кластер 3 (Определённый): ['Потребительский кредит', 'Рефинансирование', 'Автокредит'] с оценками [0.2250657081604004, 0.3100571036338806, 0.20571684837341309]
Кластер 4 (Определённый): ['Автокредит', 'Рефинансирование ипотеки', 'Потребительский кредит'] с оценками [0.20571684837341309, 0.18984343111515045, 0.2250657081604004]
Кластер 5 (Определённый): ['Кредиты наличными', 'Потребительский кредит', 'Кредитная карта'] с оценками [0.27568092942237854, 0.2250657081604004, 0.25561290979385376]
Кластер 6 (О

In [95]:
for cluster_id, info in cluster_to_categories.items():
    print(f"Кластер {cluster_id} ({info['status']}): {info['top_categories']} с оценками {info['top_scores']}")

Кластер 0 (Определённый): ['Кредиты наличными', 'Потребительский кредит', 'Кредитная карта'] с оценками [0.8941195011138916, 0.8810106515884399, 0.8775705099105835]
Кластер 1 (Определённый): ['Кредитная карта', 'Дебетовая карта', 'Пенсионная карта'] с оценками [0.8506721258163452, 0.8277544975280762, 0.8235695362091064]
Кластер 2 (Определённый): ['Кредиты наличными', 'Кредитная карта', 'Потребительский кредит'] с оценками [0.8512441515922546, 0.846103310585022, 0.8354851603507996]
Кластер 3 (Определённый): ['Потребительский кредит', 'Рефинансирование', 'Автокредит'] с оценками [0.921901524066925, 0.9103912115097046, 0.9070243835449219]
Кластер 4 (Определённый): ['Автокредит', 'Рефинансирование ипотеки', 'Потребительский кредит'] с оценками [0.934280276298523, 0.9175980687141418, 0.9174801111221313]
Кластер 5 (Определённый): ['Кредиты наличными', 'Потребительский кредит', 'Кредитная карта'] с оценками [0.8905243873596191, 0.882354736328125, 0.8746414184570312]
Кластер 6 (Определённый): 

## Проверяем качество кластеризации

In [212]:
df['product'].unique()

array(['debitCards', 'serviceLevel', 'creditCards', 'credits', 'savings',
       'mortgage', 'autocredits', 'creditRefinancing',
       'mortgageRefinancing', 'currencyExchange', 'usloviya',
       'moneyOrder', 'businessRko', 'acquiring', 'remoteService', 'other',
       'creditcards', 'hypothec', 'restructing', 'deposits', 'transfers',
       'mobile_app', 'individual'], dtype=object)

In [213]:
df

,index,id,title,text,dateCreate,product,source,review,cluster_id
0,0,992651,Не выполняют условия акции!,В апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,sravni,Не выполняют условия акции! В апреле 2025 года...,28
1,1,998360,Жалоба на услугу «Газпром Бонус Премиум»,Купил услугу Газпром Бонус «Премиум» за 2 990 ...,2025-09-15T09:38:13.34818Z,debitCards,sravni,Жалоба на услугу «Газпром Бонус Премиум» Купил...,15
2,2,993744,Банк не отвечает за слова своих сотрудников! Н...,"Хочу поделиться историей, которая убила моё до...",2025-09-02T23:21:16.507166Z,debitCards,sravni,Банк не отвечает за слова своих сотрудников! Н...,0
3,3,999494,Не выполняют условия акции!,В июне 2025 года я порекомендовал премиальную ...,2025-09-18T08:10:19.954986Z,debitCards,sravni,Не выполняют условия акции! В июне 2025 года я...,15
4,4,992291,Странный банк,Добрый день! В связи с устройством на новую ра...,2025-08-28T19:29:51.696341Z,debitCards,sravni,Странный банк Добрый день! В связи с устройств...,21
...,...,...,...,...,...,...,...,...,...
50364,50364,10885124,Под видом кредита 12.9% пытаются впарить креди...,Газпромбанк тут заманивал кредитом одобренным ...,2023-04-15 23:33:45,individual,banki,Под видом кредита 12.9% пытаются впарить креди...,16
50365,50365,10872392,Неполное изменение номера в офисе/ отделении,Ситуация довольно странная для входа в интерне...,2023-03-30 12:39:01,individual,banki,Неполное изменение номера в офисе/ отделении С...,24
50366,50366,10852603,Один из отличных банков,В банке нравится все. И обслуживание на отличн...,2023-03-10 15:04:12,individual,banki,Один из отличных банков В банке нравится все. ...,22
50367,50367,10852597,Обращение в контакт центр 10 баллов,"Обращение в контакт центр 10 баллов, а вот раб...",2023-03-10 14:57:02,individual,banki,Обращение в контакт центр 10 баллов Обращение ...,13


In [ ]:
topic_to_category = {
    "debitCards": "Дебетовые карты",
    "serviceLevel": "Обслуживание",
    "creditCards": "Кредитные карты",
    "credits": "Кредиты наличными",
    "savings": "Вклады",
    "mortgage": "Ипотека",
    "autocredits": "Автокредиты",
    "creditRefinancing": "Рефинансирование кредитов",
    "mortgageRefinancing": "Рефинансирование ипотеки",
    "currencyExchange": "Обмен валют",
    "usloviya": "Условия",
    "moneyOrder": "Денежные переводы",
    # "businessRko": "Обслуживание",
    # "acquiring": "Обслуживание",
    "remoteService": "Обслуживание",
    "other": "Другое",
    "creditcards": "Кредитные карты",
    "hypothec": "Ипотека",
    "restructing": "Рефинансирование кредитов",
    "deposits": "Вклады",
    "transfers": "Денежные переводы",
    "mobile_app": "Мобильное приложение",
    "individual": "Физические лица"
}


In [ ]:
mapping = {
    "debitCards": ("Дебетовые карты", "Дебетовая карта"),
    "creditCards": ("Кредитные карты", "Кредитная карта"),
    "credits": ("Кредиты наличными", "Потребительский кредит"),
    "mortgage": ("Ипотека", "Ипотека на дом"),
    "savings": ("Вклады", "Вклад"),
    "restructing": ("Рефинансирование кредитов", "Реструктуризация"),
    "serviceLevel": ("Обслуживание", "Обслуживание в офисе"),
    "autocredits": ("Автокредиты", "Автокредит"),
    "remoteService": ("Обслуживание", "Дистанционное обслуживание"),
    "other": ("Другое", "Другие услуги"),
    "individual": ("Другое", "Другое (физ. лица)"),
    "creditRefinancing": ("Рефинансирование кредитов", "Рефинансирование"),
    "moneyOrder": ("Денежные переводы", "Денежные переводы"),
    "mortgageRefinancing": ("Рефинансирование ипотеки", "Рефинансирование ипотеки"),
    "currencyExchange": ("Обмен валют", "Обмен валют"),
    "mobile_app": ("Мобильное приложение", "Мобильное приложение"),
    "usloviya": ("Условия", "Условия"),
}


## Old Variants

In [63]:
kw_model = KeyBERT(model=embedding_model)
cluster_keywords = {}

for cluster_id in range(30):
    cluster_texts = [docs[i] for i, label in enumerate(labels) if label == cluster_id]
    if not cluster_texts:
        continue
    combined_text = " ".join(cluster_texts)
    keywords = kw_model.extract_keywords(combined_text, top_n=10)
    cluster_keywords[cluster_id] = [kw for kw, score in keywords]

for cluster_id, kws in cluster_keywords.items():
    print(f"Кластер {cluster_id}: {kws}")

Кластер 0: ['обналичивание', 'задолженннсть', 'рефинансирование', 'закредитования', 'транзакций', 'транзакции', 'задолженностей', 'транзакция', 'росбанка', 'задолженнасть']
Кластер 1: ['оформляла', 'переоформила', 'обслужила', 'проконсультировала', 'рефинансировала', 'обслуживала', 'оплатила', 'переложила', 'заполняла', '88001000701']
Кластер 2: ['035000руб', 'рефинансировала', '11567руб', '1350000руб', '1200000руб', '50000руб', 'кредитная_карта', 'кредиту_953', '11600руб', '8000рублей']
Кластер 3: ['125000руб', '750000руб', '300000руб', '62000руб', '167500руб', '67000руб', '88000руб', '450000руб', '76000рублей', '000руб']
Кластер 4: ['автокредит', 'рефинансировали', 'россельхозбанка', 'банкмолчит', 'рефинансируемый', 'оформляем', 'рускредитбанка', 'платежки', 'потребкредит', 'газпромабанка']
Кластер 5: ['говнобанк', 'совкомбанк', 'газпромбанка', 'газзпромбанка', 'газпромбанк', 'аьфабанк', 'лохотрон', 'газмпромбанка', 'автокредит', 'гаспромбанк']
Кластер 6: ['halkbank', 'paypal', 'кэшб

In [ ]:
[docs[i] for i, label in enumerate(labels) if label == cluster_id]

In [56]:
umap_model = UMAP(
    n_neighbors=25,
    n_components=35,
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

hdbscan_model = HDBSCAN(
    min_cluster_size=75,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

topic_model = BERTopic(
    embedding_model=None,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=None,  # отключаем TF-IDF
    language="russian",
    verbose=True
)

In [57]:
topics, probs = topic_model.fit_transform(docs, embeddings)

2025-09-23 01:06:29,926 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-23 01:07:02,252 - BERTopic - Dimensionality - Completed ✓
2025-09-23 01:07:02,253 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-23 01:07:05,981 - BERTopic - Cluster - Completed ✓
2025-09-23 01:07:05,984 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-23 01:07:07,555 - BERTopic - Representation - Completed ✓


In [58]:
topic_info = topic_model.get_topic_info()

In [59]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,52,-1_на_не_что_по,"[на, не, что, по, url, отец, все, банка, но, он]",[Мучения с ипотекой продолжаются Наконец-то вв...
1,0,11773,0_не_на_что_по,"[не, на, что, по, банка, мне, банк, за, но, как]",[Некомпетентность работы банка в целом <ul>\r ...
2,1,6642,1_не_на_что_по,"[не, на, что, по, мне, банка, банк, за, карту,...","[Наихудший из всех банков, обходите его сторон..."
3,2,4232,2_все_на_очень_по,"[все, на, очень, по, за, не, спасибо, банка, ч...","[Газпромбанк лично для меня, лучший банк! З З ..."
4,3,517,3_мы_не_на_что,"[мы, не, на, что, нам, нас, по, банк, все, банка]","[Зачем платить 3000 руб., если мы еще не испол..."


In [51]:
kw_model = KeyBERT(model=embedding_model)

In [ ]:
cluster_keywords = {}

for cluster_id in set(topics):
    if cluster_id == -1:
        continue  # пропускаем шумовые тексты
    # берём все отзывы, которые попали в этот кластер
    cluster_texts = [reviews[i] for i, t in enumerate(topics) if t == cluster_id]
    combined_text = " ".join(cluster_texts)
    
    # извлекаем топ-10 ключевых слов/фраз
    keywords = kw_model.extract_keywords(combined_text, top_n=10)
    
    # оставим только слова/фразы
    cluster_keywords[cluster_id] = [kw for kw, score in keywords]

# Выведем ключевые слова для каждого кластера
for cluster_id, kws in cluster_keywords.items():
    print(f"Кластер {cluster_id}: {kws}")

In [9]:
topic_model = BERTopic(embedding_model=embedding_model, 
                       language="russian",    # поддержка русского в c-TF-IDF
                       verbose=True)

topics, probs = topic_model.fit_transform(docs)

2025-09-22 22:01:12,620 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1553 [00:00<?, ?it/s]

2025-09-22 22:08:16,594 - BERTopic - Embedding - Completed ✓
2025-09-22 22:08:16,595 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-22 22:08:43,651 - BERTopic - Dimensionality - Completed ✓
2025-09-22 22:08:43,653 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-22 22:08:46,369 - BERTopic - Cluster - Completed ✓
2025-09-22 22:08:46,380 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-22 22:08:50,730 - BERTopic - Representation - Completed ✓


In [11]:
topic_info = topic_model.get_topic_info()

In [12]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5995,-1_очень_спасибо_быстро_все,"[очень, спасибо, быстро, все, благодарность, б...",[Быстрая доставка карты Спасибо представителю ...
1,0,25054,0_не_на_что_по,"[не, на, что, по, банк, банка, мне, за, но, то]",[Банк не для физических лиц Я оформил карту Юн...
2,1,13389,1_не_что_на_мне,"[не, что, на, мне, по, за, банк, карту, банка,...",[Вводят в заблуждение! Добрый день! 30.01.2025...
3,2,1534,2_очень_спасибо_карту_представитель,"[очень, спасибо, карту, представитель, рассказ...",[Получение дебетовой карты Union Pay Заказала ...
4,3,515,3_мы_нам_нас_ипотеку,"[мы, нам, нас, ипотеку, документы, что, на, кв...",[Некомпетентность специалистов банка Добрый де...
...,...,...,...,...,...
86,85,12,85_спасибо_попал_сотруднице_подсказало,"[спасибо, попал, сотруднице, подсказало, опера...","[Благодарность Обратился в данный офис, для по..."
87,86,11,86_спасибо_очень_пользованию_помогла,"[спасибо, очень, пользованию, помогла, девушка...",[Спасибо сотруднику Благодарю сотрудницу Банка...
88,87,10,87_успешное_выигрышем_нудного_запоминают,"[успешное, выигрышем, нудного, запоминают, про...",[Очень успешное сотрудничество Возникла идея р...
89,88,10,88_являюсь_клиентом_доволен_достаточно,"[являюсь, клиентом, доволен, достаточно, банко...",[Обслуживание Мне банк понравился. Являюсь кли...


In [ ]:


# --- 6. Ключевые слова по кластерам ---
for topic_num in topic_info["Topic"].head(10):
    if topic_num != -1:  # -1 это "шум"
        print(f"Тема {topic_num}: {topic_model.get_topic(topic_num)}")